# LSTM Models for Regression

In [1]:
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.io import output_notebook, show


np.random.seed(17)
output_notebook()

Using TensorFlow backend.
/Users/gvbgduh/Projects/DNN_models/env/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Loading BokehJS ...

### Design some artificial data

In [2]:
param = 20.
small_periods = 200
scale = 1.1
pick_freq = 10
dataset_1 = np.arange(param)
dataset_2 = np.arange(param)
dataset_3 = np.arange(param)

for i in range(1, small_periods):
    # Simple case
    d_1 = (np.arange(param) + (pick_freq * i  / 2)) * (scale + i / 50)
    dataset_1 = np.concatenate((dataset_1, d_1))
    # Interesting case
    d_2 = (np.arange(param) + (pick_freq * np.sin(i))) * (scale + i / 1)
    dataset_2 = np.concatenate((dataset_2, d_2))
    # Another interesting one
    d_3 = (np.arange(param) + (pick_freq * (i / 2) * np.sin(i))) * (scale + i / 100)
    dataset_3 = np.concatenate((dataset_3, d_3))


print(dataset_1.shape)
print(dataset_2.shape)
print(dataset_3.shape)

(4000,)
(4000,)
(4000,)


In [3]:
p_1 = figure(title='Dataset 1', plot_height=400, plot_width=900)
p_1.line(np.arange(len(dataset_1)), dataset_1, line_width=1)
p_2 = figure(title='Dataset 2', plot_height=400, plot_width=900)
p_2.line(np.arange(len(dataset_2)), dataset_2, line_width=1)
p_3 = figure(title='Dataset 3', plot_height=400, plot_width=900)
p_3.line(np.arange(len(dataset_3)), dataset_3, line_width=1)
show(column(p_1, p_2, p_3))

### Prepare data

In [4]:
scaler_1 = MinMaxScaler(feature_range=(0, 1))
scaler_2 = MinMaxScaler(feature_range=(0, 1))
scaler_3 = MinMaxScaler(feature_range=(0, 1))

ds_1 = scaler_1.fit_transform(dataset_1.reshape(-1, 1))
ds_2 = scaler_2.fit_transform(dataset_2.reshape(-1, 1))
ds_3 = scaler_3.fit_transform(dataset_3.reshape(-1, 1))

# scaler.inverse_transform(array) for inverse

In [ ]:
# Double check
p_1 = figure(title='Dataset 1 Scaled', plot_height=400, plot_width=900)
p_1.line(np.arange(len(ds_1)), ds_1.reshape(-1), line_width=1)
p_2 = figure(title='Dataset 2 Scaled', plot_height=400, plot_width=900)
p_2.line(np.arange(len(ds_2)), ds_2.reshape(-1), line_width=1)
p_3 = figure(title='Dataset 3 Scaled', plot_height=400, plot_width=900)
p_3.line(np.arange(len(ds_3)), ds_3.reshape(-1), line_width=1)
show(column(p_1, p_2, p_3))

In [5]:
def create_input(data, window_size=1):
    x, y = [], []
    for i in range(len(data) - window_size - 1):
        x.append(data[i:i + window_size, 0])
        y.append(data[i + window_size, 0])
    x, y = np.array(x), np.array(y)
    # Reshape input to [samples, timesteps, features]
    return x.reshape(x.shape[0], 1, x.shape[1]), y

def split_data(data, percentage=0.8):
    idx = int(len(data) * percentage)
    return data[:idx,:], data[idx:,:]

In [6]:
def get_plot_data(dataset, train, test, window_size):
    print('Incoming shapes: {}, {}, {}'.format(dataset.shape, train.shape, test.shape))
    train_plot = np.empty_like(dataset)
    train_plot[:, :] = np.nan
    train_plot[window_size:len(train)+window_size, :] = train
    # shift test predictions for plotting
    test_plot = np.empty_like(dataset)
    test_plot[:, :] = np.nan
    test_plot[len(train)+(window_size*2)+1:len(dataset)-1, :] = test
    x_axis = np.arange(len(dataset))
    print('Outcoming shapes: {}, {}, {}'.format(dataset.shape, train_plot.shape, test_plot.shape))
    return (
        (x_axis, dataset.reshape(-1)),
        (x_axis, train_plot.reshape(-1)),
        (x_axis, test_plot.reshape(-1)),
    )

def show_error(train, test):
    train_error = np.sqrt(mean_squared_error(train[0].reshape(-1), train[1].reshape(-1)))
    print('Train Score: %.2f RMSE' % (train_error))
    test_error = np.sqrt(mean_squared_error(test[0].reshape(-1), test[1].reshape(-1)))
    print('Test Score: %.2f RMSE' % (test_error))

def show_result(data, name):
    p = figure(title=name, plot_height=400, plot_width=900)
    p.line(data[0][0], data[0][1], line_width=1, color='blue', alpha=0.6)
    p.line(data[1][0], data[1][1], line_width=1, color='orange', alpha=0.6)
    p.line(data[2][0], data[2][1], line_width=2, color='green', alpha=0.6)
    show(p)

## Model 1 for Dataset 1 - Shallow LSTM

In [37]:
batch_size_1 = 32
window_size_1 = 1
timesteps_1 = 1

train_1, test_1 = split_data(ds_1)
train_1_x, train_1_y = create_input(train_1, window_size_1)
test_1_x, test_1_y = create_input(test_1, window_size_1)

print(
    """
    Train shape: {}, test shape: {}
    Train input shape: {}, train output shape {}
    Test input shape {}, test output shape {}
    """.format(train_1.shape, test_1.shape,
               train_1_x.shape, train_1_y.shape,
               test_1_x.shape, test_1_y.shape))


    Train shape: (3200, 1), test shape: (800, 1)
    Train input shape: (3198, 1, 1), train output shape (3198,)
    Test input shape (798, 1, 1), test output shape (798,)
    


In [38]:
model_1 = Sequential()
model_1.add(LSTM(128, input_shape=(1, window_size_1)))
model_1.add(Dense(1))
model_1.compile(
    loss='mse', 
    optimizer='adam',
)

history_1 = model_1.fit(
    train_1_x, train_1_y,
    batch_size=batch_size_1, epochs=50, shuffle=False,
)

Epoch 1/50
3198/3198 [==============================] - 2s 648us/step - loss: 5.4706e-06
Epoch 2/50
3198/3198 [==============================] - 1s 160us/step - loss: 0.0145
Epoch 3/50
3198/3198 [==============================] - 1s 170us/step - loss: 0.0050
Epoch 4/50
3198/3198 [==============================] - 1s 167us/step - loss: 0.0011
Epoch 5/50
3198/3198 [==============================] - 1s 173us/step - loss: 2.1043e-04
Epoch 6/50
3198/3198 [==============================] - 1s 166us/step - loss: 5.5046e-05
Epoch 7/50
3198/3198 [==============================] - 1s 163us/step - loss: 2.7433e-05
Epoch 8/50
3198/3198 [==============================] - 1s 187us/step - loss: 1.9374e-05
Epoch 9/50
3198/3198 [==============================] - 1s 208us/step - loss: 1.5908e-05
Epoch 10/50
3198/3198 [==============================] - 1s 162us/step - loss: 1.4036e-05
Epoch 11/50
3198/3198 [==============================] - 1s 200us/step - loss: 1.2939e-05
Epoch 12/50
3198/3198 [========

In [39]:
train_predict_1 = model_1.predict(train_1_x)
test_predict_1 = model_1.predict(test_1_x)
print(train_predict_1.shape, test_predict_1.shape)

train_predict_1 = scaler_1.inverse_transform(train_predict_1)
test_predict_1 = scaler_1.inverse_transform(test_predict_1)
print(train_predict_1.shape, test_predict_1.shape)

train_1_y_unscaled = scaler_1.inverse_transform([train_1_y])
test_1_y_unscaled = scaler_1.inverse_transform([test_1_y])
print(train_1_y_unscaled.shape, test_1_y_unscaled.shape)

(3198, 1) (798, 1)
(3198, 1) (798, 1)
(1, 3198) (1, 798)


In [40]:
show_error((train_1_y_unscaled, train_predict_1), (test_1_y_unscaled, test_predict_1))
show_result(
    get_plot_data(dataset_1.reshape(-1,1), 
                  train_predict_1, 
                  test_predict_1, window_size_1),
    'Model 1')

Train Score: 14.16 RMSE
Test Score: 13.67 RMSE
Incoming shapes: (4000, 1), (3198, 1), (798, 1)
Outcoming shapes: (4000, 1), (4000, 1), (4000, 1)


## Model 2 for Dataset 2 - Stacked LSTM

In [71]:
batch_size_2 = 64
window_size_2 = 64
timesteps_2 = 1

train_2, test_2 = split_data(ds_2)
train_2_x, train_2_y = create_input(train_2, window_size_2)
test_2_x, test_2_y = create_input(test_2, window_size_2)

print(
    """
    Train shape: {}, test shape: {}
    Train input shape: {}, train output shape {}
    Test input shape {}, test output shape {}
    """.format(train_2.shape, test_2.shape,
               train_2_x.shape, train_2_y.shape,
               test_2_x.shape, test_2_y.shape))


    Train shape: (3200, 1), test shape: (800, 1)
    Train input shape: (3135, 1, 64), train output shape (3135,)
    Test input shape (735, 1, 64), test output shape (735,)
    


In [72]:
model_2 = Sequential()
model_2.add(LSTM(512, return_sequences=True, input_shape=(1, window_size_2)))
model_2.add(LSTM(512, return_sequences=True))
model_2.add(LSTM(512, return_sequences=True))
model_2.add(LSTM(512))
model_2.add(Dense(1))
model_2.compile(
    loss='mse', 
    optimizer='adam',
)

history_2 = model_2.fit(
    train_2_x, train_2_y,
    batch_size=batch_size_2, epochs=1700, shuffle=False,
)

Epoch 1/1700
3135/3135 [==============================] - 20s 6ms/step - loss: 0.0156
Epoch 2/1700
3135/3135 [==============================] - 14s 4ms/step - loss: 0.0032
Epoch 3/1700
3135/3135 [==============================] - 13s 4ms/step - loss: 0.0037
Epoch 4/1700
3135/3135 [==============================] - 13s 4ms/step - loss: 0.0040
Epoch 5/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 0.0038
Epoch 6/1700
3135/3135 [==============================] - 14s 5ms/step - loss: 0.0039
Epoch 7/1700
3135/3135 [==============================] - 14s 5ms/step - loss: 0.0036
Epoch 8/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 0.0040
Epoch 9/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 0.0058
Epoch 10/1700
3135/3135 [==============================] - 14s 4ms/step - loss: 0.0048
Epoch 11/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 0.0095
Epoch 12/1700
3135/3135 [===========================

3135/3135 [==============================] - 12s 4ms/step - loss: 1.2457e-04
Epoch 94/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 1.2358e-04
Epoch 95/1700
3135/3135 [==============================] - 14s 5ms/step - loss: 1.1554e-04
Epoch 96/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 1.0668e-04
Epoch 97/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 9.7564e-05
Epoch 98/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 9.1757e-05
Epoch 99/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 9.0678e-05
Epoch 100/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 8.7971e-05
Epoch 101/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 8.4492e-05
Epoch 102/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 8.3608e-05
Epoch 103/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 7.6225e-05
Epoch 104

3135/3135 [==============================] - 12s 4ms/step - loss: 4.3035e-05
Epoch 183/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.8076e-05
Epoch 184/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 5.4941e-05
Epoch 185/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 4.9070e-05
Epoch 186/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.8947e-05
Epoch 187/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.8286e-05
Epoch 188/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 4.8943e-05
Epoch 189/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 4.3361e-05
Epoch 190/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 4.3590e-05
Epoch 191/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.8195e-05
Epoch 192/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 4.2292e-05
Epo

3135/3135 [==============================] - 12s 4ms/step - loss: 1.2723e-04
Epoch 272/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 5.5611e-05
Epoch 273/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.4732e-05
Epoch 274/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.8940e-05
Epoch 275/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.2812e-05
Epoch 276/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.1957e-05
Epoch 277/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.8968e-05
Epoch 278/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.1569e-05
Epoch 279/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.1660e-05
Epoch 280/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.5544e-05
Epoch 281/1700
3135/3135 [==============================] - 13s 4ms/step - loss: 3.1206e-05
Epo

3135/3135 [==============================] - 12s 4ms/step - loss: 2.5204e-05
Epoch 361/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.7443e-05
Epoch 362/1700
3135/3135 [==============================] - 14s 4ms/step - loss: 2.8653e-05
Epoch 363/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.2738e-05
Epoch 364/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 1.9893e-05
Epoch 365/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.4782e-05
Epoch 366/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.0225e-05
Epoch 367/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.5175e-05
Epoch 368/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 1.5398e-05
Epoch 369/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 1.4742e-05
Epoch 370/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 1.6782e-05
Epo

...

3135/3135 [==============================] - 12s 4ms/step - loss: 3.0242e-05
Epoch 1336/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.7159e-05
Epoch 1337/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.1469e-05
Epoch 1338/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 1.7426e-05
Epoch 1339/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 1.1219e-05
Epoch 1340/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 7.9835e-06
Epoch 1341/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 6.0543e-06
Epoch 1342/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 5.6249e-06
Epoch 1343/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 5.5593e-06
Epoch 1344/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 5.9097e-06
Epoch 1345/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 6.10

3135/3135 [==============================] - 12s 4ms/step - loss: 2.7630e-05
Epoch 1424/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.6443e-05
Epoch 1425/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 4.3929e-05
Epoch 1426/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.6255e-05
Epoch 1427/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.4947e-05
Epoch 1428/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.0593e-05
Epoch 1429/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.6179e-05
Epoch 1430/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 1.6918e-05
Epoch 1431/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 9.1428e-06
Epoch 1432/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 4.7605e-06
Epoch 1433/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.10

3135/3135 [==============================] - 12s 4ms/step - loss: 1.8317e-05
Epoch 1512/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 9.1504e-06
Epoch 1513/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 4.5547e-06
Epoch 1514/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 5.8866e-06
Epoch 1515/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 6.2832e-06
Epoch 1516/1700
3135/3135 [==============================] - 13s 4ms/step - loss: 6.6827e-06
Epoch 1517/1700
3135/3135 [==============================] - 14s 5ms/step - loss: 6.6660e-06
Epoch 1518/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 5.5716e-06
Epoch 1519/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 4.7147e-06
Epoch 1520/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.3998e-06
Epoch 1521/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.03

3135/3135 [==============================] - 12s 4ms/step - loss: 7.4462e-06
Epoch 1600/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.1014e-05
Epoch 1601/1700
3135/3135 [==============================] - 13s 4ms/step - loss: 2.6138e-05
Epoch 1602/1700
3135/3135 [==============================] - 13s 4ms/step - loss: 1.3506e-05
Epoch 1603/1700
3135/3135 [==============================] - 13s 4ms/step - loss: 1.8216e-05
Epoch 1604/1700
3135/3135 [==============================] - 13s 4ms/step - loss: 1.6984e-05
Epoch 1605/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 6.5369e-06
Epoch 1606/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 9.1382e-06
Epoch 1607/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.6341e-06
Epoch 1608/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 4.7998e-06
Epoch 1609/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.83

3135/3135 [==============================] - 12s 4ms/step - loss: 2.1129e-06
Epoch 1688/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.1601e-06
Epoch 1689/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.3219e-06
Epoch 1690/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.5797e-06
Epoch 1691/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.7624e-06
Epoch 1692/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.7867e-06
Epoch 1693/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.7089e-06
Epoch 1694/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.6781e-06
Epoch 1695/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 2.8413e-06
Epoch 1696/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 3.5039e-06
Epoch 1697/1700
3135/3135 [==============================] - 12s 4ms/step - loss: 4.67

In [73]:
train_predict_2 = model_2.predict(train_2_x)
test_predict_2 = model_2.predict(test_2_x)
print(train_predict_2.shape, test_predict_2.shape)

train_predict_2 = scaler_2.inverse_transform(train_predict_2)
test_predict_2 = scaler_2.inverse_transform(test_predict_2)
print(train_predict_2.shape, test_predict_2.shape)

train_2_y_unscaled = scaler_2.inverse_transform([train_2_y])
test_2_y_unscaled = scaler_2.inverse_transform([test_2_y])
print(train_2_y_unscaled.shape, test_2_y_unscaled.shape)

(3135, 1) (735, 1)
(3135, 1) (735, 1)
(1, 3135) (1, 735)


In [74]:
show_error((train_2_y_unscaled, train_predict_2), (test_2_y_unscaled, test_predict_2))
show_result(
    get_plot_data(dataset_2.reshape(-1,1), 
                  train_predict_2, 
                  test_predict_2, window_size_2),
    'Model 2')

Train Score: 66.73 RMSE
Test Score: 103.31 RMSE
Incoming shapes: (4000, 1), (3135, 1), (735, 1)
Outcoming shapes: (4000, 1), (4000, 1), (4000, 1)


In [75]:
from keras.models import load_model

model_2.save('model_2.h2')

For 256x3 B-32 W-1 E-50
Train Score: 448.38 RMSE
Test Score: 796.60 RMSE

B-64
Train Score: 442.07 RMSE
Test Score: 796.38 RMSE

E - 300
Train Score: 434.92 RMSE
Test Score: 798.18 RMSE

256x4 E-50
Train Score: 447.34 RMSE
Test Score: 797.42 RMSE

W-16
Train Score: 432.92 RMSE
Test Score: 780.39 RMSE

W-64
Train Score: 350.90 RMSE
Test Score: 348.58 RMSE

512x4 E-10
Train Score: 775.75 RMSE
Test Score: 418.81 RMSE

E-1700
Train Score: 66.73 RMSE
Test Score: 103.31 RMSE